## Load libraries


In [14]:
import pandas as pd
import numpy as np
import os
import random
import sklearn
from sklearn.impute import SimpleImputer
import seaborn as sb
from scipy import stats
from sklearn.ensemble import IsolationForest
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.datasets import dump_svmlight_file
from xgboost import XGBRegressor
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.feature_selection import VarianceThreshold
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression, SelectFromModel
from sklearn.ensemble import ExtraTreesRegressor
from scipy.stats import pearsonr
import lightgbm as lgb



X = pd.read_csv(os.path.join("data", "X_train.csv"),
                delimiter=',',
                index_col = 'id')
y = pd.read_csv(os.path.join("data", "y_train.csv"),
                delimiter=',',
                index_col='id')


# idx = []
# for i, feature in enumerate(X.columns):
#     values = X[feature].dropna(0)
#     mx = values.max()
#     mn = values.min()
#     if mn > 0 and abs((mx - mn) / mn - .1) < 0.05:
#         idx.append(feature)
# X = X.drop(idx, axis=1)
# print(X.shape)
# nan_idx = np.isnan(X)



nan_idx = np.isnan(X)
imputer = KNNImputer(n_neighbors=20)
X_imp = imputer.fit_transform(X)


X_train = X_imp
y_train = y.to_numpy()



# model = sklearn.ensemble.IsolationForest(max_samples=0.5, contamination=0.025)
# # model = sklearn.neighbors.LocalOutlierFactor(contamination=0.025)
# outl_pred = model.fit_predict(X_imp)
# mask = outl_pred != -1
# X_cleaned, y_cleaned = X_cleaned[mask, :], y_cleaned[mask]
# print(X_cleaned.shape, y_cleaned.shape)
# y_train = y_cleaned
X_train.shape

(1212, 832)

In [15]:
thresh_corr = 0.1

df_train = pd.DataFrame(X_train, columns=X.columns)
feat = df_train.iloc[:,1:].columns.tolist()
corr = {}
for f in feat:
    x_f = df_train[[f]][f].values
    key = f
    corr[key] = pearsonr(x_f,y_train)[0]

data_corr = pd.DataFrame(corr, index=['lin']).T
selected_feat = data_corr.loc[(abs(data_corr['lin']) >= thresh_corr) ].index
selected_feat


/scratch_net/troubadour/tianfwang/conda_envs/pytorch3d/lib/python3.9/site-packages/scipy/stats/_stats_py.py:4068: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/scratch_net/troubadour/tianfwang/conda_envs/pytorch3d/lib/python3.9/site-packages/scipy/stats/_stats_py.py:4068: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/scratch_net/troubadour/tianfwang/conda_envs/pytorch3d/lib/python3.9/site-packages/scipy/stats/_stats_py.py:4068: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/scratch_net/troubadour/tianfwang/conda_envs/pytorch3d/lib/python3.9/site-packages/scipy/stats/_stats_py.py:4068: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not d

Index(['x2', 'x15', 'x21', 'x23', 'x26', 'x27', 'x29', 'x40', 'x69', 'x77',
       ...
       'x778', 'x780', 'x783', 'x788', 'x796', 'x801', 'x817', 'x819', 'x823',
       'x824'],
      dtype='object', length=189)

In [16]:
X_train_copy = X_train.copy()

X_train = sklearn.preprocessing.StandardScaler().fit(X_train_copy).transform(X_train)
X_train  = pd.DataFrame(X_train, columns=X.columns)

X_train = X_train[selected_feat].values

In [18]:
from sklearn.feature_selection import RFE

params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'num_leaves': 32,
    'learning_rate': 0.05,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}
model = lgb.LGBMRegressor(**params)

selector = RFE(model, step=0.1, n_features_to_select=60, verbose=1)

In [19]:
selector.fit(X_train,y_train.ravel())


Fitting estimator with 189 features.
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001686 seconds.
You can set `force_col_wise=true` to remove the overhead.
Fitting estimator with 171 features.
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_byt

RFE(estimator=LGBMRegressor(bagging_fraction=0.8, bagging_freq=5,
                            feature_fraction=0.5, learning_rate=0.05,
                            num_leaves=32, objective='regression', task='train',
                            verbose=0),
    n_features_to_select=60, step=0.1, verbose=1)

In [6]:
X_train.shape

(1212, 189)

In [20]:
X_cleaned, y_cleaned = X_train, y_train

X_train = selector.transform(X_train)


model = sklearn.ensemble.IsolationForest(contamination="auto")
outl_pred = model.fit_predict(X_train)
mask = outl_pred != -1

X_train, y_train = X_train[mask, :], y_train[mask]



In [8]:
X_train.shape

(1147, 120)

In [21]:
# X_train_backup = X_train.copy()
# X_train = sklearn.preprocessing.StandardScaler().fit(X_train_backup).transform(X_train)

X_test = pd.read_csv(os.path.join("data", "X_test.csv"),
                     delimiter=',',
                     index_col='id')
X_test_backup = X_test
# X_test = X_test.drop(idx, axis=1)
X_test = imputer.fit(X_imp).transform(X_test)
X_test_stdised = sklearn.preprocessing.StandardScaler().fit(X_train_copy).transform(X_test)



/scratch_net/troubadour/tianfwang/conda_envs/pytorch3d/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but KNNImputer was fitted without feature names
  warnings.warn(


In [22]:
X_test_stdised = pd.DataFrame(X_test_stdised, columns=X_test_backup.columns, index=X_test_backup.index)
X_test_stdised = X_test_stdised[selected_feat].values

X_test_stdised = selector.transform(X_test_stdised)

In [1]:
def r2_sc(preds, train_data):
    labels = train_data.get_label()
    return 'r2', r2_score(labels, preds), True

best = 0
trial_itr = 100
for i in range(trial_itr):
    print(f"train itr {i}")
    X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train, y_train, test_size = 0.15) 
    
    lgb_train = lgb.Dataset(X_train_split, y_train_split)
    
    
    lgb_eval = lgb.Dataset(X_val_split, y_val_split, reference=lgb_train)


    params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'num_leaves': 32,
        'learning_rate': 0.05,
        'feature_fraction': 0.5,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'verbose': 0
    }



    model = lgb.train(params,
                    lgb_train,
                    num_boost_round=1000,
                    feval=r2_sc,
                    valid_sets={lgb_train, lgb_eval},
                     early_stopping_rounds=100)
    y_val_predict = model.predict(X_val_split, num_iteration=model.best_iteration)
        
    
    
    res = r2_score(y_val_split, y_val_predict)
    
    print(res)
    if res > best:
        print(f"New Best result {res}")
        best = res
        y_pred = model.predict(X_test_stdised, num_iteration=model.best_iteration)
        f = open(f"out{best}.csv", "w")
        f.write("id,y\n")
        for i,x in enumerate(y_pred):
            f.write("{},{}\n".format(i,x))
        f.close()

train itr 0


NameError: name 'train_test_split' is not defined

In [ ]:
import glob
print(glob.glob("./test/*"))

In [13]:
import glob

inputs = glob.glob("./esb/*")
print(inputs)
res = [0.0]*776

for f in inputs:
    f = open(f).readlines()[1:]
    for l in f:
        sp = l.split(',')
        idx, s = int(sp[0]), float(sp[1])
        res[idx] += s
out = open("./ensemble_debug.csv", "w")
out.write("id,y\n")
for i in range(776):
    out.write("{},{}\n".format(i, (res[i]/len(inputs))))
out.close()

['./esb/out0.7342315896063321.csv', './esb/out0.7346987687926609.csv', './esb/out0.7372855502427849.csv', './esb/out0.7247752664242797.csv']
